In [1]:
import numpy as np

from math import sqrt, log, floor, gamma

def _higuchi_fd(data, kmax=10):
    """Utility function for :func:`compute_higuchi_fd`.
    Parameters
    ----------
    data : ndarray, shape (n_channels, n_times)
    kmax : int
    Returns
    -------
    output : ndarray, shape (n_channels,)
    """
    n_times = data.size
    lk = np.empty((kmax,))
    x_reg = np.empty((kmax,))
    y_reg = np.empty((kmax,))
    for k in range(1, kmax + 1):
        lm = np.empty((k,))
        for m in range(k):
            ll = 0
            n_max = floor((n_times - m - 1) / k)
            n_max = int(n_max)
            for j in range(1, n_max):
                ll += abs(data[m + j * k] - data[m + (j - 1) * k])
            ll /= k
            ll *= (n_times - 1) / (k * n_max)
            lm[m] = ll
        # Mean of lm
        m_lm = 0
        for m in range(k):
            m_lm += lm[m]
        m_lm /= k
        lk[k - 1] = m_lm
        x_reg[k - 1] = log(1. / k)
        y_reg[k - 1] = log(m_lm)
    # Compute slope of linear regression
    ssxm, ssxym, _, _ = np.cov(x_reg, y_reg, bias=1).flat
    return np.true_divide(ssxym, ssxm)

In [2]:
np.random.seed(123456)
x = np.random.rand(3000)

print(_higuchi_fd(x, kmax=10))

%timeit _higuchi_fd(x, kmax=10)

1.9954220134363372
17.1 ms ± 216 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [3]:
from mne_features import univariate as unv
print(unv.compute_higuchi_fd(x[np.newaxis, ...]))
%timeit unv.compute_higuchi_fd(x[np.newaxis, ...])

[1.99542201]
75.6 µs ± 1.13 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
